In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

dados = pd.read_csv("/content/sample_data/SolarPrediction.csv")

limiar = dados["Radiation"].median()
dados["Alvo"] = (dados["Radiation"] > limiar).astype(int)

try:
    dados["Hora"] = pd.to_datetime(dados["Time"], format="%H:%M:%S").dt.hour
except Exception:
    pass

colunas_remover = ["Radiation","Alvo","Data","Time","TimeSunRise","TimeSunSet"]
colunas_remover = [c for c in colunas_remover if c in dados.columns]
X = dados.drop(columns=colunas_remover)

X = X.select_dtypes(include=[np.number]).copy()
y = dados["Alvo"].copy()

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

escalador = StandardScaler()
X_treino = escalador.fit_transform(X_treino)
X_teste  = escalador.transform(X_teste)

modelos = {
    "Árvore de Decisão": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=300),
    "SVM (RBF)": SVC(kernel="rbf", random_state=42)
}

resultados = {}
for nome, modelo in modelos.items():
    modelo.fit(X_treino, y_treino)
    y_pred = modelo.predict(X_teste)

    acuracia = accuracy_score(y_teste, y_pred)
    matriz   = confusion_matrix(y_teste, y_pred)
    resultados[nome] = {"acuracia": acuracia, "matriz": matriz}

    print("="*50)
    print(f"Modelo: {nome}")
    print(f"Acurácia: {acuracia:.4f}")
    print(classification_report(y_teste, y_pred, target_names=["Baixa","Alta"], zero_division=0))
    print("Matriz de Confusão:\n", matriz)

print("\nResumo das Acurácias:")
for nome, res in resultados.items():
    print(f"{nome}: {res['acuracia']:.4f}")


Modelo: Árvore de Decisão
Acurácia: 0.9695
              precision    recall  f1-score   support

       Baixa       0.97      0.97      0.97      4906
        Alta       0.97      0.97      0.97      4900

    accuracy                           0.97      9806
   macro avg       0.97      0.97      0.97      9806
weighted avg       0.97      0.97      0.97      9806

Matriz de Confusão:
 [[4778  128]
 [ 171 4729]]
Modelo: Random Forest
Acurácia: 0.9761
              precision    recall  f1-score   support

       Baixa       0.98      0.98      0.98      4906
        Alta       0.98      0.98      0.98      4900

    accuracy                           0.98      9806
   macro avg       0.98      0.98      0.98      9806
weighted avg       0.98      0.98      0.98      9806

Matriz de Confusão:
 [[4789  117]
 [ 117 4783]]
Modelo: SVM (RBF)
Acurácia: 0.9586
              precision    recall  f1-score   support

       Baixa       0.95      0.97      0.96      4906
        Alta       0.97 